Install the latest OpenAI package...

In [1]:
!pip install openai --upgrade

In [2]:
import os
from openai import OpenAI
client = OpenAI()

Upload our training and evaluation files, in chat completion format:

In [8]:
client.files.create(
  file=open("./DATA_train.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-9lI2ovFA1UJskgOPpxDTwEhG at 0x2266872ea90> JSON: {
  "object": "file",
  "id": "file-9lI2ovFA1UJskgOPpxDTwEhG",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 1774941,
  "created_at": 1692794707,
  "status": "uploaded",
  "status_details": null
}

In [9]:
client.files.create(
  file=open("./DATA_eval.jsonl", "rb"),
  purpose='fine-tune'
)

<File file id=file-UqPVnkk9z8Q74BEUqPlnhjHL at 0x226669e59f0> JSON: {
  "object": "file",
  "id": "file-UqPVnkk9z8Q74BEUqPlnhjHL",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 442619,
  "created_at": 1692794711,
  "status": "uploaded",
  "status_details": null
}

Check the status of these files by copying in the returned ID's above. If there are JSON errors they will be reported here.

In [11]:
client.files.retrieve("file-UqPVnkk9z8Q74BEUqPlnhjHL")

<File file id=file-UqPVnkk9z8Q74BEUqPlnhjHL at 0x2266865c180> JSON: {
  "object": "file",
  "id": "file-UqPVnkk9z8Q74BEUqPlnhjHL",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 442619,
  "created_at": 1692794711,
  "status": "uploaded",
  "status_details": null
}

Start our fine tuning job! Copy in the ID's for our uploaded training and validation files.

In [12]:
client.fine_tuning.jobs.create(training_file="file-9lI2ovFA1UJskgOPpxDTwEhG", validation_file="file-UqPVnkk9z8Q74BEUqPlnhjHL", model="gpt-3.5-turbo")

<FineTuningJob fine_tuning.job id=ftjob-mQlhbPB5vsog1SeDLNx2xAMj at 0x226669e58b0> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-mQlhbPB5vsog1SeDLNx2xAMj",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692794886,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-DBeDgDH8c36NSJobwuaBPXrW",
  "result_files": [],
  "status": "created",
  "validation_file": "file-UqPVnkk9z8Q74BEUqPlnhjHL",
  "training_file": "file-9lI2ovFA1UJskgOPpxDTwEhG",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

Get general info about this job.

In [13]:
client.fine_tuning.jobs.retrieve("ftjob-mQlhbPB5vsog1SeDLNx2xAMj")

<FineTuningJob fine_tuning.job id=ftjob-mQlhbPB5vsog1SeDLNx2xAMj at 0x22666a16220> JSON: {
  "object": "fine_tuning.job",
  "id": "ftjob-mQlhbPB5vsog1SeDLNx2xAMj",
  "model": "gpt-3.5-turbo-0613",
  "created_at": 1692794886,
  "finished_at": null,
  "fine_tuned_model": null,
  "organization_id": "org-DBeDgDH8c36NSJobwuaBPXrW",
  "result_files": [],
  "status": "running",
  "validation_file": "file-UqPVnkk9z8Q74BEUqPlnhjHL",
  "training_file": "file-9lI2ovFA1UJskgOPpxDTwEhG",
  "hyperparameters": {
    "n_epochs": 3
  },
  "trained_tokens": null
}

Monitor its progress. When done, you can use the resulting fine tuned model ID in the playground (or the API)

In [32]:
client.fine_tuning.jobs.list_events(id="ftjob-mQlhbPB5vsog1SeDLNx2xAMj", limit=10)

<OpenAIObject list at 0x22668759950> JSON: {
  "object": "list",
  "data": [
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-n0GA9lmPtAulghPIgsfSSdM1",
      "created_at": 1692797270,
      "level": "info",
      "message": "Fine-tuning job successfully completed",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-miblzvSktANikUk7sJOQe6Ir",
      "created_at": 1692797267,
      "level": "info",
      "message": "New fine-tuned model created: ft:gpt-3.5-turbo-0613:sundog-software-llc::7qiBf2gI",
      "data": null,
      "type": "message"
    },
    {
      "object": "fine_tuning.job.event",
      "id": "ftevent-VVGJVV3Ss6N6ROQjK4xhib6T",
      "created_at": 1692796566,
      "level": "info",
      "message": "Step 1000: training loss=1.89",
      "data": {
        "step": 1000,
        "train_loss": 1.8927490711212158,
        "train_mean_token_accuracy": 0.5443037748336792
      },
      "type": "

For comparison, see how the non-fine-tuned GPT model does:

In [30]:
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "Data is an android in the TV series Star Trek: The Next Generation."},
    {"role": "user", "content": "PICARD: Mr. Data, scan for lifeforms."}
  ]
)

print(completion.choices[0].message)


{
  "role": "assistant",
  "content": "DATA: Aye, Captain. Scanning for lifeforms. However, as an android, I do not possess the biological sensors typically used for such scans. Instead, I rely on my advanced sensory systems and tricorder readings to detect signs of life.\n\n(Pauses briefly)\n\nDATA: I am detecting several lifeforms in the vicinity, Captain. Shall I provide detailed analysis?"
}


When it's done, try our fine-tuned model! Copy in our fine tuned ID.

In [31]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0613:sundog-software-llc::7qiBf2gI",
  messages=[
    {"role": "system", "content": "Data is an android in the TV series Star Trek: The Next Generation."},
    {"role": "user", "content": "PICARD: Mr. Data, scan for lifeforms."}
  ]
)

print(completion.choices[0].message)

{
  "role": "assistant",
  "content": "DATA: I am finding very low levels of radioactivity -- I cannot make a conclusive scan for lifeforms, Captain."
}
